In [48]:
#!/usr/bin/env python3
import math
import numpy as np
import warnings
import sys
#from scipy.spatial.transform import Rotation
import os 
import general_robotics_toolbox as rox
import time
from Arm_Lib import Arm_Device

# Get DOFBOT object
Arm = Arm_Device()
time.sleep(.1)

def Dofbot():
    #inital the dofbot parameters
    
    x=np.array([1,0,0])
    y=np.array([0,1,0])
    z=np.array([0,0,1])
    a=np.array([0,0,0])
    
    H = np.array([z,-y,-y,-y,-x]).T    
    P = np.array([0.1045*z, a, 0.08285*x, -0.08285*z, a, -0.12842*x]).T
    joint_type=[0,0,0,0,0]

    
    return rox.Robot(H, P, joint_type)    


def inverse_K(Rd, Pd, q0, alpha,  tol=[0.02,0.02,0.02,0.001,0.001,0.001], Nmax=200):
    n = len(q0)
    q = np.zeros(n)
    q = q0
    P0t = np.zeros(3)
    iternum = 0
    fwd = rox.fwdkin(Dofbot(),q0)#do the forward kinematics
    R = fwd.R
    P = fwd.p
    dR = R @ np.transpose(Rd)
    k = np.transpose(rox.R2rpy(dR))  
    dX =  np.concatenate((k, (P-Pd)),axis = None)
    while np.prod(abs(dX)).all()>0.02:
            #check maximum internum
        if iternum <= Nmax:
            fwd = rox.fwdkin(Dofbot(),q)
            R = fwd.R
            P = fwd.p
            #get jacobian matrix
            Jq = rox.robotjacobian(Dofbot(),q)
            dR = R @ np.transpose(Rd)
            k = np.transpose(rox.R2rpy(dR))
            dX =  np.concatenate((k, (P-Pd)),axis = None)
            #update the q
            q=q-alpha*np.linalg.pinv(Jq)@dX
            iternum = iternum+1
            #print(q)
        else:
            break
    return q, P0t, iternum

def path_angle_generate(x,y,z):
    #input values
    Rd = [[-0.75, -0.1047, -0.6531],
          [-0.433, 0.8241, 0.3652],
          [0.5   , 0.5567, -0.6634]]
    Pd = [x,
          y,
          z]
    q0 = np.deg2rad(np.transpose([25, 50, 75, 30, 30]))

    a,b,c = inverse_K(Rd, Pd, q0, alpha=0.3,  tol=[0.02,0.02,0.02,0.001,0.001,0.001], Nmax=200)
    sa = np.rad2deg(a)
    return sa


if __name__ == "__main__":
    #input values
    angle = path_angle_generate(0.2058, 0.1188, 0.1464)
    print(angle)
    Arm.Arm_serial_servo_write6(angle[0], angle[1], angle[2], angle[3], angle[4], 90, 500)
    time.sleep(1)
    

[30.00087374 45.11811687 79.78275984 25.10027228 40.00109383]
